In [1]:
import nltk
import os
import re
from nltk.corpus import shakespeare
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from flask import Flask, render_template, request

app = Flask(__name__, static_url_path='/static')
app.secret_key = 'key'

NLPDoc = shakespeare.words('hamlet.xml')
for i in range(len(NLPDoc)):
    NLPDoc[i] = NLPDoc[i].lower()

punctuation = re.compile(r'[-.?!,:;()|0-9]')
post_punctuation = []
for words in NLPDoc:
    word = punctuation.sub("",words)
    if len(word)>0:
        post_punctuation.append(word)
        
new_token = []
for word in post_punctuation:
    if word not in stopwords.words('english'):
        new_token.append(word)

@app.route('/')
@app.route('/home')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():    
    check = request.form['sentence']
    check_tokens = word_tokenize(check)
    
    check_last = check_tokens[len(check_tokens)-1]
    check_idx = len(check_last)
    
    res = [idx for idx in new_token if idx[0:check_idx] == check_last.lower()]  
    
    distinct_count = FreqDist()
    for word in res:
        distinct_count[word.lower()]+=1
    result = distinct_count.most_common(10)
    
    check_tokens[len(check_tokens)-1] = result[0][0]
    
    check_str_final = TreebankWordDetokenizer().detokenize(check_tokens)
    
    return render_template('index.html', result=result, check_str=check_str_final)

if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Aug/2021 10:50:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2021 10:50:05] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Aug/2021 10:50:05] "GET /static/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Aug/2021 10:50:05] "GET /static/bootstrap/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [07/Aug/2021 10:50:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Aug/2021 10:50:13] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [07/Aug/2021 10:50:13] "GET /static/bootstrap/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Aug/2021 10:50:13] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Aug/2021 10:50:13] "GET /static/bootstrap/js/bootstrap.min.js HTTP/1.1" 304 -
